# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17384271305742104024
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357433344
locality {
  bus_id: 1
}
incarnation: 16345272605041992067
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
            #Equivalent to sum(english_words_counter.values())
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    
    #Deciphering Code with character-level RNNs class
    #https://classroom.udacity.com/nanodegrees/nd892/parts/94230570-1456-442c-a152-ca4e171acbfc/modules/162cd048-bbff-412e-8f14-29c34cc2c960/lessons/06c89f82-2fd1-409e-9d09-e785d32036d7/concepts/46fd995b-27ce-4553-b9c4-8bca3d0ffa3a
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)

#Show list of Words with its respective number in the tokenizer
print(text_tokenizer.word_index)

print()

#Sent: First element of the tuple, sentence
#token_sent: second element of the tuple, sentence written as numbers from tokenizer
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [8]:
#Undertanding zip function used above

print("text_sentences")
print()
print(text_sentences)

print("----------------------")

print("text_tokenized")
print()
print(text_tokenized)

print("----------------------")
print("----------------------")
print("----------------------")

zip(text_sentences,text_tokenized)

test = zip(text_sentences,text_tokenized)  # zip the values
for values in test:
    print(values)  # print each tuples
    
#It pairs sentences and its vector containing word's numbers from the tokenizer

text_sentences

['The quick brown fox jumps over the lazy dog .', 'By Jove , my quick study of lexicography won a prize .', 'This is a short sentence .']
----------------------
text_tokenized

[[1, 2, 4, 5, 6, 7, 1, 8, 9], [10, 11, 12, 2, 13, 14, 15, 16, 3, 17], [18, 19, 3, 20, 21]]
----------------------
----------------------
----------------------
('The quick brown fox jumps over the lazy dog .', [1, 2, 4, 5, 6, 7, 1, 8, 9])
('By Jove , my quick study of lexicography won a prize .', [10, 11, 12, 2, 13, 14, 15, 16, 3, 17])
('This is a short sentence .', [18, 19, 3, 20, 21])


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """

    # TODO: Implement
    
    #Followuing solution given by udacity in classroom excercise
    #https://classroom.udacity.com/nanodegrees/nd892/parts/94230570-1456-442c-a152-ca4e171acbfc/modules/162cd048-bbff-412e-8f14-29c34cc2c960/lessons/06c89f82-2fd1-409e-9d09-e785d32036d7/concepts/46fd995b-27ce-4553-b9c4-8bca3d0ffa3a
    #pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)
    
    if length is None:
        length = max([len(sentence) for sentence in x])
        
    padded = pad_sequences(x, maxlen = length, padding='post')

    return padded
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [12]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    #Sources:
    #Based on class excercise resolution
    #https://classroom.udacity.com/nanodegrees/nd892/parts/94230570-1456-442c-a152-ca4e171acbfc/modules/162cd048-bbff-412e-8f14-29c34cc2c960/lessons/06c89f82-2fd1-409e-9d09-e785d32036d7/concepts/46fd995b-27ce-4553-b9c4-8bca3d0ffa3a
    #and
    #https://github.com/bkktimber/AIND-Machine-Translation/blob/master/machine_translation.ipynb
    
    #Hyperparameters
    learning_rate = 0.001

    #Equivalence of names between class resolution and bkktimber's github to make it more understanable!
    #input_seq = inputs; rnn = hidden_layer, logits = outputs
    inputs = Input(input_shape[1:])
    hidden_layer = GRU(units = output_sequence_length, return_sequences=True)(inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(hidden_layer)

    #model = None
    model = Model(inputs = inputs, outputs = outputs)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size + 1)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=30, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 19s 173us/step - loss: 3.2220 - acc: 0.4241 - val_loss: 2.5314 - val_acc: 0.4690
Epoch 2/30
110288/110288 [==============================] - 17s 153us/step - loss: 2.4450 - acc: 0.4779 - val_loss: 2.3648 - val_acc: 0.4777
Epoch 3/30
110288/110288 [==============================] - 17s 155us/step - loss: 2.2686 - acc: 0.4858 - val_loss: 2.1531 - val_acc: 0.5036
Epoch 4/30
110288/110288 [==============================] - 18s 160us/step - loss: 2.0500 - acc: 0.5240 - val_loss: 1.9545 - val_acc: 0.5539
Epoch 5/30
110288/110288 [==============================] - 18s 159us/step - loss: 1.8797 - acc: 0.5602 - val_loss: 1.8035 - val_acc: 0.5676
Epoch 6/30
110288/110288 [==============================] - 17s 150us/step - loss: 1.7520 - acc: 0.5748 - val_loss: 1.7117 - val_acc: 0.5807
Epoch 7/30
110288/110288 [==============================] - 16s 144us/step - loss: 1.6818 - acc: 0.5826

In [13]:
#TIRAR

# Print prediction(s)
print("Prediction:")
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois parfois en l' et il il est en en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


# Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [14]:
#model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    #Sources:
    #Based on class excercise resolution
    #https://classroom.udacity.com/nanodegrees/nd892/parts/94230570-1456-442c-a152-ca4e171acbfc/modules/162cd048-bbff-412e-8f14-29c34cc2c960/lessons/803aa80a-5caa-4fc9-934f-3357d1593616/concepts/6f3626f0-0917-4efc-ab52-74d26269b929
    #and
    #https://github.com/bkktimber/AIND-Machine-Translation/blob/master/machine_translation.ipynb
    
    #Hyperparameters
    learning_rate = 0.001

    #Equivalence of names between class resolution and bkktimber's github to make it more understanable!
    #Between the () its always the name of the object in the line before
    #input_dim = vocabulary_size, output_dim = embedding_size
    inputs = Input(input_shape[1:]) 
        #Embedding(input_dim, output_dim, embeddings_initializer='uniform',
        #embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)
    embedding_layer = Embedding(input_dim = english_vocab_size, output_dim = output_sequence_length)(inputs)
    hidden_layer = GRU(units = output_sequence_length, return_sequences = True)(embedding_layer)
    outputs = TimeDistributed(Dense(french_vocab_size, activation = 'softmax'))(hidden_layer)
       
    #model = None
    model = Model(inputs = inputs, outputs = outputs)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
    
    
    # TODO: Implement
    return None
tests.test_embed_model(embed_model)

# TODO: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
    #Shape now is different from before, we lose the number 1 in the end.
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size + 1)

#TROCAR
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=30, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 17s 153us/step - loss: 3.3737 - acc: 0.4071 - val_loss: 2.5328 - val_acc: 0.4458
Epoch 2/30
110288/110288 [==============================] - 16s 143us/step - loss: 1.8753 - acc: 0.5711 - val_loss: 1.4777 - val_acc: 0.6406
Epoch 3/30
110288/110288 [==============================] - 17s 158us/step - loss: 1.2795 - acc: 0.6865 - val_loss: 1.1151 - val_acc: 0.7261
Epoch 4/30
110288/110288 [==============================] - 18s 166us/step - loss: 1.0228 - acc: 0.7483 - val_loss: 0.9393 - val_acc: 0.7684
Epoch 5/30
110288/110288 [==============================] - 18s 164us/step - loss: 0.8840 - acc: 0.7781 - val_loss: 0.8284 - val_acc: 0.7878
Epoch 6/30
110288/110288 [==============================] - 17s 152us/step - loss: 0.7924 - acc: 0.7957 - val_loss: 0.7564 - val_acc: 0.8026
Epoch 7/30
110288/110288 [==============================] - 16s 150us/step - loss: 0.7292 - acc: 0.8093

In [15]:
#TIRAR

# Print prediction(s)
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois calme en l' et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [16]:
#model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
        
    #Source:
    #Based on class excercise resolution
    #https://github.com/bkktimber/AIND-Machine-Translation/blob/master/machine_translation.ipynb
    
    #Hyperparameters
    learning_rate = 0.001
  
    #We go back to Model 1 but input Bidirectional option in the hidden layer
    inputs = Input(input_shape[1:]) 
    layer = GRU(units = output_sequence_length, return_sequences = True)
        #keras.layers.Bidirectional(layer, merge_mode='concat', weights=None)
    hidden_layer = Bidirectional(layer)(inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation = 'softmax'))(hidden_layer)
       
    #model = None
    model = Model(inputs = inputs, outputs = outputs)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
    
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
    #Shape now is just like the one used in model 1
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size + 1)

bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=30, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 26s 233us/step - loss: 2.8917 - acc: 0.4771 - val_loss: 2.0979 - val_acc: 0.5478
Epoch 2/30
110288/110288 [==============================] - 27s 242us/step - loss: 1.8502 - acc: 0.5695 - val_loss: 1.6998 - val_acc: 0.5826
Epoch 3/30
110288/110288 [==============================] - 27s 249us/step - loss: 1.6288 - acc: 0.5923 - val_loss: 1.5703 - val_acc: 0.6010
Epoch 4/30
110288/110288 [==============================] - 28s 258us/step - loss: 1.5265 - acc: 0.6081 - val_loss: 1.4868 - val_acc: 0.6147
Epoch 5/30
110288/110288 [==============================] - 28s 252us/step - loss: 1.4540 - acc: 0.6202 - val_loss: 1.4220 - val_acc: 0.6262
Epoch 6/30
110288/110288 [==============================] - 28s 252us/step - loss: 1.3982 - acc: 0.6297 - val_loss: 1.3756 - val_acc: 0.6342
Epoch 7/30
110288/110288 [==============================] - 28s 253us/step - loss: 1.3570 - acc: 0.6367

In [17]:
#TIRAR

# Print prediction(s)
print("Prediction:")
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois humide en en et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [18]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    #Sources:
    #https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
    #https://github.com/bkktimber/AIND-Machine-Translation/blob/master/machine_translation.ipynb
    #https://keras.io/getting-started/sequential-model-guide/
    
    #Hyperparameters
    learning_rate = 0.001
    units = 512
    #Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
        
        ## Define an input sequence and process it.
        #encoder_inputs = Input(shape=(None, num_encoder_tokens))
        #encoder = LSTM(latent_dim, return_state=True)
        #encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        ## We discard `encoder_outputs` and only keep the states.
        #encoder_states = [state_h, state_c]

    #Encoder
    encoder_inputs = Input(shape=input_shape[1:])
    encoder = GRU(output_sequence_length)(encoder_inputs)
    encoder_outputs = Dense(units = units, activation='relu')(encoder)
    
    
        ######
        ## Set up the decoder, using `encoder_states` as initial state.
        #decoder_inputs = Input(shape=(None, num_decoder_tokens))
        #decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
        #decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
        #decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        #decoder_outputs = decoder_dense(decoder_outputs)

        ## Define the model that will turn
        ## `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
        #model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
        ######  

    #Decoder
                #keras.layers.RepeatVector(n)
    decoder_inputs = RepeatVector(output_sequence_length)(encoder_outputs)
    decoder_gru = GRU(units = units, return_sequences=True)(decoder_inputs)
    decoder_dense = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    decoder_outputs = decoder_dense(decoder_gru)

    #Create Model from parameters defined above
    model = Model(inputs = encoder_inputs, outputs = decoder_outputs)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    #return None
    return model
tests.test_encdec_model(encdec_model)

# OPTIONAL: Train and Print prediction(s)
# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
    #Shape like model 1
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    english_vocab_size + 1,
    french_vocab_size + 1)

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=30, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 50s 452us/step - loss: 2.3304 - acc: 0.4998 - val_loss: 1.7950 - val_acc: 0.5560
Epoch 2/30
110288/110288 [==============================] - 49s 445us/step - loss: 1.6122 - acc: 0.5752 - val_loss: 1.4690 - val_acc: 0.5995
Epoch 3/30
110288/110288 [==============================] - 49s 444us/step - loss: 1.4058 - acc: 0.6112 - val_loss: 1.3428 - val_acc: 0.6266
Epoch 4/30
110288/110288 [==============================] - 49s 446us/step - loss: 1.3294 - acc: 0.6260 - val_loss: 1.2963 - val_acc: 0.6322
Epoch 5/30
110288/110288 [==============================] - 50s 449us/step - loss: 1.2966 - acc: 0.6306 - val_loss: 1.2762 - val_acc: 0.6346
Epoch 6/30
110288/110288 [==============================] - 49s 448us/step - loss: 1.2822 - acc: 0.6326 - val_loss: 1.2843 - val_acc: 0.6316
Epoch 7/30
110288/110288 [==============================] - 49s 445us/step - loss: 1.2625 - acc: 0.6350

In [19]:
#TIRAR

# Print prediction(s)
print("Prediction:")
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois agréable en l' mais il est il en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [20]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    #Hyperparameters
    learning_rate = 0.001
            #Increase units to 256 or 512
    units = 512
    #units = 32
    
    ##New
    inputs = Input(input_shape[1:]) 
    embedding_layer = Embedding(input_dim = english_vocab_size, output_dim = output_sequence_length)(inputs)
    layer = LSTM(units = output_sequence_length)
    bdlayer = Bidirectional(layer)(embedding_layer)
    encoder_outputs = Dense(units = units, activation='relu')(bdlayer)
    #Decoder
    decoder_inputs = RepeatVector(output_sequence_length)(encoder_outputs)
    decoder_gru = LSTM(units = units, return_sequences=True)
    decoder_bdlayer = Bidirectional(decoder_gru)(decoder_inputs)
    decoder_dense = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    decoder_outputs = decoder_dense(decoder_bdlayer)
    ##
    
    ##Before
    #inputs = Input(input_shape[1:]) 
    #embedding_layer = Embedding(input_dim = english_vocab_size, output_dim = output_sequence_length)(inputs)
    #layer = GRU(units = output_sequence_length)
    #bdlayer = Bidirectional(layer)(embedding_layer)
    #encoder_outputs = Dense(units = units, activation='relu')(bdlayer)
    #Decoder
    #decoder_inputs = RepeatVector(output_sequence_length)(encoder_outputs)
    #decoder_gru = GRU(units = units, return_sequences=True)
    #decoder_bdlayer = Bidirectional(decoder_gru)(decoder_inputs)
    #decoder_dense = TimeDistributed(Dense(2 * french_vocab_size, activation='softmax'))
    #decoder_outputs = decoder_dense(decoder_bdlayer)
    ##
            
    
    #Create Model from parameters defined above
    model = Model(inputs = inputs, outputs = decoder_outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_model_final(model_final)

print('Final Model Loaded')

# TODO: Train the final model

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
    #Shape like model 1
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
final_model = model_final(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    english_vocab_size + 1,
    french_vocab_size + 1)

final_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=30, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 125s 1ms/step - loss: 1.8208 - acc: 0.5671 - val_loss: 1.1769 - val_acc: 0.6767
Epoch 2/30
110288/110288 [==============================] - 123s 1ms/step - loss: 1.0193 - acc: 0.7038 - val_loss: 0.8941 - val_acc: 0.7289
Epoch 3/30
110288/110288 [==============================] - 123s 1ms/step - loss: 0.8062 - acc: 0.7503 - val_loss: 0.7293 - val_acc: 0.7710
Epoch 4/30
110288/110288 [==============================] - 123s 1ms/step - loss: 0.6784 - acc: 0.7859 - val_loss: 0.6245 - val_acc: 0.8006
Epoch 5/30
110288/110288 [==============================] - 123s 1ms/step - loss: 0.5732 - acc: 0.8175 - val_loss: 0.5349 - val_acc: 0.8298
Epoch 6/30
110288/110288 [==============================] - 123s 1ms/step - loss: 0.4959 - acc: 0.8413 - val_loss: 0.4495 - val_acc: 0.8576
Epoch 7/30
110288/110288 [==============================] - 123s 1ms/step - loss: 0.4241 -

In [21]:
#TIRAR

# Print prediction(s)
print("Prediction:")
print(logits_to_text(final_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


## Prediction (IMPLEMENTATION)

In [23]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    
    #We should use same lines we werer using before, modifing only the parameters, once they changed names
    #source: https://github.com/bkktimber/AIND-Machine-Translation/blob/master/machine_translation.ipynb
    model = model_final(x.shape,
                        y.shape[1],
                        len(x_tk.word_index) + 1,
                        len(y_tk.word_index) + 1)
    
    model.fit(x, y, batch_size=256, epochs=30, validation_split=0.2)

    #
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 119s 1ms/step - loss: 1.7788 - acc: 0.5732 - val_loss: 1.1378 - val_acc: 0.6880
Epoch 2/30
110288/110288 [==============================] - 117s 1ms/step - loss: 0.9527 - acc: 0.7223 - val_loss: 0.8179 - val_acc: 0.7515
Epoch 3/30
110288/110288 [==============================] - 117s 1ms/step - loss: 0.7371 - acc: 0.7730 - val_loss: 0.6472 - val_acc: 0.7984
Epoch 4/30
110288/110288 [==============================] - 117s 1ms/step - loss: 0.5791 - acc: 0.8175 - val_loss: 0.5420 - val_acc: 0.8264
Epoch 5/30
110288/110288 [==============================] - 117s 1ms/step - loss: 0.4574 - acc: 0.8531 - val_loss: 0.4233 - val_acc: 0.8607
Epoch 6/30
110288/110288 [==============================] - 118s 1ms/step - loss: 0.3621 - acc: 0.8804 - val_loss: 0.3341 - val_acc: 0.8887
Epoch 7/30
110288/110288 [==============================] - 117s 1ms/step - loss: 0.2890 - acc: 0.9033 - val_



## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [24]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 412967 bytes to machine_translation.html',
 '[NbConvertApp] Converting notebook machine_translation-Original.ipynb to html',
 '[NbConvertApp] Writing 331290 bytes to machine_translation-Original.html',
 '[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html',
 '[NbConvertApp] Writing 328614 bytes to machine_translation-zh.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?